In [1]:
from datetime import datetime
import matplotlib.pyplot as plt
import os
import re
import unicodedata

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, when, isnull, expr
from pyspark.sql.types import StringType, IntegerType

#### Setting variables

In [2]:
path_wd   = os.getcwd()
path_root = path_wd.rsplit("\\",0)[0]


path_raw = path_root + '\\raw\\'

path_raw_bancos = path_raw + 'bancos\\'
path_raw_empregados = path_raw + 'empregados\\'
path_raw_reclamacoes = path_raw + 'reclamacoes\\'

flist_raw_bancos = os.listdir(path_raw_bancos)
flist_raw_empregados = os.listdir(path_raw_empregados)
flist_raw_reclamacoes = os.listdir(path_raw_reclamacoes)


path_stage = path_root + '\\stage\\'

path_stage_bancos = path_stage + 'bancos\\'
path_stage_empregados = path_stage + 'empregados\\'
path_stage_reclamacoes = path_stage + 'reclamacoes\\'

path_raw

'c:\\Users\\Diane\\git\\grupo1-ingestao-dados\\Atividade 4 - Python PySpark\\raw\\'

#### Initialize SparkSession

In [3]:
spark = SparkSession.builder.appName("DataExtraction").getOrCreate()

#### 1. Data Extraction

1.1. Bancos

In [4]:
fname_bancos = path_raw_bancos + flist_raw_bancos[0]

df_bancos = spark.read.option("delimiter", ",").option("header", True).option("charset", "utf-8").csv(fname_bancos)

1.2. Empregados

In [5]:
fname_empregados_01 = path_raw_empregados + flist_raw_empregados[0]
fname_empregados_02 = path_raw_empregados + flist_raw_empregados[1]

df_empregados_01 = spark.read.option("delimiter", "|").option("header", True).csv(fname_empregados_01)
df_empregados_02 = spark.read.option("delimiter", "|").option("header", True).csv(fname_empregados_02)

1.3. Reclamações

In [6]:
fname_reclamacoes_202101 = path_raw_reclamacoes + flist_raw_reclamacoes[0]
fname_reclamacoes_202102 = path_raw_reclamacoes + flist_raw_reclamacoes[1]
fname_reclamacoes_202103 = path_raw_reclamacoes + flist_raw_reclamacoes[2]
fname_reclamacoes_202104 = path_raw_reclamacoes + flist_raw_reclamacoes[3]
fname_reclamacoes_202201 = path_raw_reclamacoes + flist_raw_reclamacoes[4]
fname_reclamacoes_202202 = path_raw_reclamacoes + flist_raw_reclamacoes[5]
fname_reclamacoes_202203 = path_raw_reclamacoes + flist_raw_reclamacoes[6]
fname_reclamacoes_202204 = path_raw_reclamacoes + flist_raw_reclamacoes[7]

df_reclamacoes_202101 = spark.read.option("delimiter", ";").option("header", True).csv(fname_reclamacoes_202101)
df_reclamacoes_202102 = spark.read.option("delimiter", ";").option("header", True).csv(fname_reclamacoes_202102)
df_reclamacoes_202103 = spark.read.option("delimiter", ";").option("header", True).csv(fname_reclamacoes_202103)
df_reclamacoes_202104 = spark.read.option("delimiter", ";").option("header", True).csv(fname_reclamacoes_202104)
df_reclamacoes_202201 = spark.read.option("delimiter", ";").option("header", True).csv(fname_reclamacoes_202201)
df_reclamacoes_202202 = spark.read.option("delimiter", ";").option("header", True).csv(fname_reclamacoes_202202)
df_reclamacoes_202203 = spark.read.option("delimiter", ";").option("header", True).csv(fname_reclamacoes_202203)
df_reclamacoes_202204 = spark.read.option("delimiter", ";").option("header", True).csv(fname_reclamacoes_202204)

#### 2. Rename columns

2.1. Bancos

In [7]:
column_to_rename_old = ['Segmento','CNPJ','Nome']
column_to_rename_new = ['segmento','cnpj','nome']

for old, new in zip(column_to_rename_old, column_to_rename_new):
    df_bancos = df_bancos.withColumnRenamed(old, new)

2.2. Empregados

In [8]:
column_to_rename_old = ['employer_name', 'reviews_count', 'culture_count', 'salaries_count', 'benefits_count', 'employer-website', 'employer-headquarters', 'employer-founded', 'employer-industry', 'employer-revenue', 'url', 'Geral', 'Cultura e valores', 'Diversidade e inclusÃ£o', 'Qualidade de vida', 'Alta lideranÃ§a', 'RemuneraÃ§Ã£o e benefÃ\xadcios', 'Oportunidades de carreira', 'Recomendam para outras pessoas(%)', 'Perspectiva positiva da empresa(%)', 'Segmento', 'CNPJ', 'Nome', 'match_percent,,,']
column_to_rename_new = ['employer_name', 'reviews_count', 'culture_count', 'salaries_count', 'benefits_count', 'employer_website', 'employer_headquarters', 'employer_founded', 'employer_industry', 'employer_revenue', 'url', 'geral', 'cultura_e_valores', 'diversidade_e_inclusao', 'qualidade_de_vida', 'alta_lideranca', 'remuneracao_e_beneficios', 'oportunidades_de_carreira', 'recomendam_para_outras_pessoas_perc', 'perspectiva_positiva_da_empresa_perc', 'segmento', 'cnpj', 'nome', 'match_percent']

for old, new in zip(column_to_rename_old, column_to_rename_new):
    df_empregados_01 = df_empregados_01.withColumnRenamed(old, new)
    df_empregados_02 = df_empregados_02.withColumnRenamed(old, new)

2.3. Reclamações

In [9]:
column_to_rename_old = ['Ano', 'Trimestre', 'Categoria', 'Tipo', 'CNPJ IF', 'Instituição financeira', 'Índice', 'Quantidade de reclamações reguladas procedentes', 'Quantidade de reclamações reguladas - outras', 'Quantidade de reclamações não reguladas', 'Quantidade total de reclamações', 'Quantidade total de clientes – CCS e SCR', 'Quantidade de clientes – CCS', 'Quantidade de clientes – SCR']
column_to_rename_new = ['ano', 'trimestre', 'categoria', 'tipo', 'cnpj_if', 'instituicao_financeira', 'indice', 'quantidade_de_reclamacoes_reguladas_procedentes', 'quantidade_de_reclamacoes_reguladas_outras', 'quantidade_de_reclamacoes_nao_reguladas', 'quantidade_total_de_reclamacoes', 'quantidade_total_de_clientes_ccs_e_scr', 'quantidade_de_clientes_ccs', 'quantidade_de_clientes_scr']

for old, new in zip(column_to_rename_old, column_to_rename_new):
    df_reclamacoes_202101 = df_reclamacoes_202101.withColumnRenamed(old, new)
    df_reclamacoes_202102 = df_reclamacoes_202102.withColumnRenamed(old, new)
    df_reclamacoes_202103 = df_reclamacoes_202103.withColumnRenamed(old, new)
    df_reclamacoes_202104 = df_reclamacoes_202104.withColumnRenamed(old, new)
    df_reclamacoes_202201 = df_reclamacoes_202201.withColumnRenamed(old, new)
    df_reclamacoes_202203 = df_reclamacoes_202203.withColumnRenamed(old, new)
    df_reclamacoes_202204 = df_reclamacoes_202204.withColumnRenamed(old, new)

#### 3. Output

In [10]:
def rename_file(fname):
    fname = fname.replace('.csv', '_extracted.csv')
    fname = fname.replace('\\', '/')

    return fname

3.1. Bancos

In [11]:
fname_bancos = rename_file(path_stage_bancos + flist_raw_bancos[0])

df_bancos.write.csv(fname_bancos, header=True, sep=";", mode="overwrite")

print(fname_bancos)

c:/Users/Diane/git/grupo1-ingestao-dados/Atividade 4 - Python PySpark/stage/bancos/EnquadramentoInicia_v2_extracted.csv


3.2. Empregados

In [12]:
fname_empregados_01 = rename_file(path_stage_empregados + flist_raw_empregados[0])
fname_empregados_02 = rename_file(path_stage_empregados + flist_raw_empregados[1])

df_empregados_01.write.csv(fname_empregados_01, header=True, sep=";", mode="overwrite")
df_empregados_02.write.csv(fname_empregados_02, header=True, sep=";", mode="overwrite")

print(fname_empregados_01)
print(fname_empregados_02)

c:/Users/Diane/git/grupo1-ingestao-dados/Atividade 4 - Python PySpark/stage/empregados/glassdoor_consolidado_join_match_less_v2_extracted.csv
c:/Users/Diane/git/grupo1-ingestao-dados/Atividade 4 - Python PySpark/stage/empregados/glassdoor_consolidado_join_match_v2_extracted.csv


3.3. Reclamações

In [13]:
fname_reclamacoes_202101 = rename_file(path_stage_reclamacoes + flist_raw_reclamacoes[0])
fname_reclamacoes_202102 = rename_file(path_stage_reclamacoes + flist_raw_reclamacoes[1])
fname_reclamacoes_202103 = rename_file(path_stage_reclamacoes + flist_raw_reclamacoes[2])
fname_reclamacoes_202104 = rename_file(path_stage_reclamacoes + flist_raw_reclamacoes[3])
fname_reclamacoes_202201 = rename_file(path_stage_reclamacoes + flist_raw_reclamacoes[4])
fname_reclamacoes_202203 = rename_file(path_stage_reclamacoes + flist_raw_reclamacoes[6])
fname_reclamacoes_202204 = rename_file(path_stage_reclamacoes + flist_raw_reclamacoes[7])

df_reclamacoes_202101.write.csv(fname_reclamacoes_202101, header=True, sep=";", mode="overwrite")
df_reclamacoes_202102.write.csv(fname_reclamacoes_202102, header=True, sep=";", mode="overwrite")
df_reclamacoes_202103.write.csv(fname_reclamacoes_202103, header=True, sep=";", mode="overwrite")
df_reclamacoes_202104.write.csv(fname_reclamacoes_202104, header=True, sep=";", mode="overwrite")
df_reclamacoes_202201.write.csv(fname_reclamacoes_202201, header=True, sep=";", mode="overwrite")
df_reclamacoes_202203.write.csv(fname_reclamacoes_202203, header=True, sep=";", mode="overwrite")
df_reclamacoes_202204.write.csv(fname_reclamacoes_202204, header=True, sep=";", mode="overwrite")

print(fname_reclamacoes_202101)
print(fname_reclamacoes_202102)
print(fname_reclamacoes_202103)
print(fname_reclamacoes_202104)
print(fname_reclamacoes_202201)
print(fname_reclamacoes_202203)
print(fname_reclamacoes_202204)

c:/Users/Diane/git/grupo1-ingestao-dados/Atividade 4 - Python PySpark/stage/reclamacoes/2021_tri_01_extracted.csv
c:/Users/Diane/git/grupo1-ingestao-dados/Atividade 4 - Python PySpark/stage/reclamacoes/2021_tri_02_extracted.csv
c:/Users/Diane/git/grupo1-ingestao-dados/Atividade 4 - Python PySpark/stage/reclamacoes/2021_tri_03_extracted.csv
c:/Users/Diane/git/grupo1-ingestao-dados/Atividade 4 - Python PySpark/stage/reclamacoes/2021_tri_04_extracted.csv
c:/Users/Diane/git/grupo1-ingestao-dados/Atividade 4 - Python PySpark/stage/reclamacoes/2022_tri_01_extracted.csv
c:/Users/Diane/git/grupo1-ingestao-dados/Atividade 4 - Python PySpark/stage/reclamacoes/2022_tri_03_extracted.csv
c:/Users/Diane/git/grupo1-ingestao-dados/Atividade 4 - Python PySpark/stage/reclamacoes/2022_tri_04_extracted.csv


In [14]:
spark.stop()